<a href="https://colab.research.google.com/github/csce585-mlsystems/CSCE585ProjectROI/blob/main/MedHive_NER_RE_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 7.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm') #Loads pretrained English model
sentence = "Why Apple is looking at buying U.K. startup for $1 billion ?"
doc = nlp(sentence)

for ent in doc.ents: #contains all recognized texts
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 4 9 ORG
U.K. 31 35 GPE
$1 billion 48 58 MONEY


In [ ]:
#Show case sensitivity
sentence = "Why apple is now looking at buying U.K. startup for $1 billion?"
doc = nlp(sentence)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

U.K. 35 39 GPE
$1 billion 52 62 MONEY


In [ ]:
from spacy.tokens import Span # to manually add a new named enitity

# Process the sentence with nlp to create a Doc object
sentence = "Ritedose is planning to launch a new medical device"
doc = nlp(sentence) # Create a Doc object from the sentence

custom_label = "ORG"
# Now pass the Doc object to Span
doc.ents = [Span(doc, 0, 1, label=custom_label)] #what does this line do?

for ent in doc.ents:
    print(ent.text,ent.label_) #doc.ents update the output

Ritedose ORG


Here we are implementing the REBEL Relationship Extraction

*   Pipeline
*   Model & Tokenizer



In [ ]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')
# We need to use the tokenizer manually since we need special tokens.
extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor("Punta Cana is a resort town in the municipality of Higuey, in La Altagracia Province, the eastern most province of the Dominican Republic", return_tensors=True, return_text=False)[0]["generated_token_ids"]])
print(extracted_text[0])
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets
extracted_triplets = extract_triplets(extracted_text[0])
print(extracted_triplets)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

Device set to use cpu


<s><triplet> Punta Cana <subj> La Altagracia Province <obj> located in the administrative territorial entity <subj> Dominican Republic <obj> country <triplet> Higuey <subj> La Altagracia Province <obj> located in the administrative territorial entity <subj> Dominican Republic <obj> country <triplet> La Altagracia Province <subj> Dominican Republic <obj> country <triplet> Dominican Republic <subj> La Altagracia Province <obj> contains administrative territorial entity</s>
[{'head': 'Punta Cana', 'type': 'located in the administrative territorial entity', 'tail': 'La Altagracia Province'}, {'head': 'Punta Cana', 'type': 'country', 'tail': 'Dominican Republic'}, {'head': 'Higuey', 'type': 'located in the administrative territorial entity', 'tail': 'La Altagracia Province'}, {'head': 'Higuey', 'type': 'country', 'tail': 'Dominican Republic'}, {'head': 'La Altagracia Province', 'type': 'country', 'tail': 'Dominican Republic'}, {'head': 'Dominican Republic', 'type': 'contains administrative 

Within this transformer pipeline we are implementing the Babel rebel model's text to generation and store it in a variable triplet extractor  

*   Task 1: Implement Tokenizer Extractor

1. tokenizer.batch_decode: a method that converts numerical ID's into human readable text  
2. return_tensors=True : returns a str or tensor flow type value where it returns tf.Tensor objects needs to be true to have any effect
3. return_text= False : Since it is false the token value will remain in numerical ID state concatenated with the generated_token_ids str value

*   Task 2: We print the first element of our initalized array extracted text at index 0

Taks 3: Create a function that parse the generated text and extract the triplets
1. definition statement extract triplet where we pass text as a parameter
2. initalized a 1D array called triplets
3. initialized variables relation, subject, object to empty str values
4. initalized text to text.strip() removing the leading and trailing values of a str value
5. create a variable called current and set it to a str value x
6. Implement a for loop where we split and replace every <s> token in text with <pad> str value
a. if tokens equals triplet value (an empty array)
- replace the current with str t
b. if the initalized relation variable does not equal empty str
-Append the empty 1D array triplet to a str head to the subject.strip() where it removes the trailing and leading values of subject
- set the relation and subject variables to empty str so the append str can be placed inside
c. if the token equals <subj> we set the current variable to s
- if the relation does not equal to an empty str value
--we append the empty 1D array triplets to the str head and to the
subject.strip() values and the type str to the relation.strip() value, the tail str to the object.strip() value
-object will equal an empty str so we can append the values to it
d. if token equals <object> str
- set current to o str and relation to an empty str values
f.Else
- if the current variable equals str value t
--subject should increase the token to the str value subject
-if the current variable equals str value s
--object should increase the token to an empty str value object
-if current variable equals str value o
--relation should increase the empty str to the token
g. if the str variable subject does not equal an inital str value and object is not equal to the inital str value
-append the empty 1D array to the str head to the subject:strip value, type str to the sub:strip value, object str value to the value object.strip()
--return the filled array of triplets


